In [ ]:
#default_exp sources.core

# Source Interface

> The interface used by the sources for our audio data.

In [ ]:
#|export

import pandas as pd

In [ ]:
#export

class Source:
    def __init__(self, name: str):
        self.name = name